In [1]:
# -*- coding: utf-8 -*-
import os
import tensorflow as tf
%run Bi_LSTM
%run Word2Vec
import gensim
import numpy as np
from eunjeon import Mecab
import pandas as pd

C:\Users\dongc\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
W2V = Word2Vec()
mecab = Mecab()
from konlpy.tag import Okt
okt = Okt()

In [3]:
df = pd.read_csv('../data/자소서_균등.csv',index_col=0, engine='python', encoding='utf8')

In [4]:
Batch_size = 1
Vector_size = 300
Maxseq_length = 3022   ## Max length of training data
learning_rate = 0.001
lstm_units = 128
num_class = len(df.label.unique())
keep_prob = 1.0

In [5]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "BiLSTM_mecab_morphs_equal_1/BiLSTM_model.ckpt"

sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)

In [11]:
df.loc[800].jss

'의지가 강한 것이 저의 가장 큰 장점입니다.\n늘 무언가를 하고자 하면 제대로 파고듭니다. 취미든 일이든, 한번 시작할 때 제대로 해보자는 생각이 강해서 다양한 정보를 습득하고 준비하곤 합니다.\n한번은 이종격투기에 관심을 가지게 된 경험이 있습니다. 앞선 문항에서 밝힌 것처럼, 초등학교 시절부터 운동을 했다 보니 기초적인 운동신경에 자신을 가지고 있었습니다. 이종격투기 도장을 다니며 아마추어 데뷔를 준비했습니다. 하루에 2천 개의 줄넘기를 하고 스파링을 하며 운동을 했습니다. 반 년 동안 꾸준히 노력한 결과 한 아마추어 이종격투기 대회에서 8강에 들어가는 쾌거를 이뤄냈습니다. 한 달 동안 상처를 치료하고 회복하느라 굉장히 아픈 나날을 보내야 했지만 제 의지를 다시 한 번 확인할 수 있었던 기회였기 때문에 후회는 없었습니다.\n제 단점은 이런 의지가 다른 사람에게도 있는 것처럼, 억지로 끌고 가려 한다는 점입니다.\n다른 사람들도 제가 가지고 있는 만큼의 의지가 있다고 생각해, 사람들을 이끌고 무언가를 하려고 하면, 사람들이 지쳐서 불만을 제기하곤 합니다.\n한번은 경제학에 대한 교양수업을 듣는데, 조별 발표를 하게 됐습니다. 다양한 의견을 담기 위해 400장의 설문지를 돌리며 발표의 질을 높이자고 했습니다. 조원들은 다른 일도 바쁜 상황에 한 사람당 백장씩 설문지를 돌리자는 것이 말이 되냐며 반발했습니다. 결국 갈등을 빚게 됐고, 조원들의 제안방식이 마음에 들지 않았던 저는 수업을 드롭하는 극단적인 선택을 했습니다. 저는 이런 성격을 바꾸기 위해 요즘은 다른 사람을 이해하기 위해 노력하고 있습니다.'

In [6]:
def classify(jss):
    tokens = okt.nouns(jss)
    embedding = W2V.Convert2Vec('../data/word2vec_mecab_morphs2.model', tokens)
    zero_pad = W2V.Zero_padding(embedding, Batch_size, Maxseq_length, Vector_size)
    global sess
    result =  sess.run(tf.argmax(prediction,1), feed_dict = {X: zero_pad , seq_len: [len(tokens)] } ) 
    print(sess.run(prediction, feed_dict = {X: zero_pad , seq_len: [len(tokens)] } ) )
    if result == 0:
        print('기타')
    elif result == 1:
        print('경영지원')
    elif result == 2:
        print('재무회계')
    elif result == 3:
        print('응용프로그래밍')

ERROR! Session/line number was not unique in database. History logging moved to new session 164


In [7]:
while(1):
    s = input("문장을 입력하세요 : ")
    if s == '':
        break
    else:
        classify(s)

문장을 입력하세요 : 의지가 강한 것이 저의 가장 큰 장점입니다.\n늘 무언가를 하고자 하면 제대로 파고듭니다. 취미든 일이든, 한번 시작할 때 제대로 해보자는 생각이 강해서 다양한 정보를 습득하고 준비하곤 합니다.\n한번은 이종격투기에 관심을 가지게 된 경험이 있습니다. 앞선 문항에서 밝힌 것처럼, 초등학교 시절부터 운동을 했다 보니 기초적인 운동신경에 자신을 가지고 있었습니다. 이종격투기 도장을 다니며 아마추어 데뷔를 준비했습니다. 하루에 2천 개의 줄넘기를 하고 스파링을 하며 운동을 했습니다. 반 년 동안 꾸준히 노력한 결과 한 아마추어 이종격투기 대회에서 8강에 들어가는 쾌거를 이뤄냈습니다. 한 달 동안 상처를 치료하고 회복하느라 굉장히 아픈 나날을 보내야 했지만 제 의지를 다시 한 번 확인할 수 있었던 기회였기 때문에 후회는 없었습니다.\n제 단점은 이런 의지가 다른 사람에게도 있는 것처럼, 억지로 끌고 가려 한다는 점입니다.\n다른 사람들도 제가 가지고 있는 만큼의 의지가 있다고 생각해, 사람들을 이끌고 무언가를 하려고 하면, 사람들이 지쳐서 불만을 제기하곤 합니다.\n한번은 경제학에 대한 교양수업을 듣는데, 조별 발표를 하게 됐습니다. 다양한 의견을 담기 위해 400장의 설문지를 돌리며 발표의 질을 높이자고 했습니다. 조원들은 다른 일도 바쁜 상황에 한 사람당 백장씩 설문지를 돌리자는 것이 말이 되냐며 반발했습니다. 결국 갈등을 빚게 됐고, 조
[[0.2040379  0.32777327 0.27988753 0.18830132]]
경영지원
문장을 입력하세요 : [시장의 판세를 바꾸겠습니다] 대학시절 투자를 하다 돈을 잃은 적이 있습니다. 그 충격에 잃은 돈만큼의 이득을 만들어야겠다는 생각에 투자에 대하여 공부를 하기 시작하며 흥미를 얻게 되었습니다. 사람들이 열심히 모은 돈으로 투자한 이득을 얻었을 때 그와 더불어 그들의 꿈을 같이 얻는 다고 생각합니다. 고객과 상생하며 그들에게 꿈을 심어줄 수 있는 대신증권의 사원이 되겠습니다.  1) 대신증권을 

[[0.22063676 0.19935465 0.24822211 0.33178648]]
응용프로그래밍
문장을 입력하세요 : 단순히 컴퓨터가 좋아서 선택한 컴퓨터공학부였습니다. JAVA와 C언어를 중심으로 한 프로그램 개발도 해봤습니다. UNIX 등을 활용한 공부도 즐거웠습니다.  대학교 1학년 때는 SW개발 공모전에 참여했습니다. 1학년이 무엇을 할 수 있겠느냐며 추천장을 써주지 않는 교수님께 삼고초려의 자세로 추천장을 받아내기도 했습니다. 물론 이 시기의 공모전의 성적은 좋지 않았습니다. 하지만 흥미를 느끼고 해마다 참여한 결과, 3학년 때는 공모전에서 금상을 받기도 했습니다. 그만큼 흥미가 있었고, 전공수업에 충실하며 지식을 쌓아왔습니다. MY-SQL 자격증을 획득하기도 했고, 해킹과 보안에 관한 공부도 꾸준히 해왔습니다. 리서치 회사에서 아르바이트를 하면서, DB에 관한 관심이 생겼습니다. 단순한 SW개발과는 또 다른 매력이 있었습니다. 그러기에 DB에 관련된 업무를 꿈꿨습니다. 하지만 또한 컴퓨터공학이라는 전공과 자격증만으로는 부족하다는 생각이 들었습니다. 오라클 자격증을 획득하기 위해 교육을 이수 받고, OCP 자격증을 획득했습니다.  또한 DB 운영을 위해서는 DB에 대한 '이해'가 필요하다고 생각했습니다. 시간 낭비라는 주위의 만류를 버리고, 경영학을 복수전공했습니다. 회계학을 부전공으로 삼아 폭 넓게 지식을 쌓았습니다. DataBase 운영 분야는 '준비된 신입'이 존재하기 힘듭니다. 하지만 저만은 이렇게 '준비된 신입'이라 자부합니다
[[0.21657301 0.23984382 0.40496856 0.13861462]]
재무회계
문장을 입력하세요 : [기본에 충실, 협업과 소통] 저는 컴퓨터 공학부 디지털 컨텐츠 전공자로서, 업무에 초석이 될 컴퓨터에 대한 이해를 다양한 수업을 통해 익혀왔습니다. 컴퓨터 구조론, 운영체제, 알고리즘 등의 수업을 통해 이론적 지식을 쌓았고 C,C++,JAVA 언어를 배워 이론으로 배운 내용을 실제로 구현해보면서 0과